In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LayerNormalization, LeakyReLU, BatchNormalization
from tensorflow.keras.layers import LSTM
from keras.regularizers import l2
from sklearn.preprocessing import StandardScaler

In [55]:
baseline_train_data = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"]:
    baseline_train_data.append(pd.read_csv(f"LSTM_data/{coin}usdt/training_output.csv"))

baseline_train_data[0][24:]

,Open time,LogReturns,LogLow/O,LogHigh/O
24,1609632000000,-0.005369,-0.007071,0.994259
25,1609635600000,-0.003918,-0.013464,0.998239
26,1609639200000,-0.004790,-0.008341,1.000000
27,1609642800000,-0.002748,-0.005848,0.999410
28,1609646400000,0.008220,-0.005172,0.994122
...,...,...,...,...
26211,1704045600000,-0.000998,-0.003663,0.996405
26212,1704049200000,-0.002332,-0.004501,0.996734
26213,1704052800000,-0.000667,-0.004177,0.988943
26214,1704056400000,0.003997,-0.000835,0.983446


In [106]:
baseline_train_data[0].loc[9909]

Open time     1.645268e+12
LogReturns   -1.612938e-02
LogLow/O     -2.224561e-02
LogHigh/O              inf
Name: 9909, dtype: float64

In [103]:
df = baseline_train_data[0]
df[np.isinf(df["LogHigh/O"])]

,Open time,LogReturns,LogLow/O,LogHigh/O
9909,1645268400000,-0.016129,-0.022246,inf


In [101]:
# Find infinity values
for coin in baseline_train_data:
    print(coin[coin.isin([np.nan, np.inf, -np.inf]).any(axis=1)])

          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  LogHigh/O
9909  1645268400000   -0.016129 -0.022246        inf
          Open time  LogReturns  LogLow/O  Log

In [84]:
X_train, y_train = [], []
for df in baseline_train_data:
    values = df[["LogReturns", "LogLow/O", "LogHigh/O"]].values
    for i in range(24, len(values)):
        X_train.append(values[i-24:i])
        y_train.append(values[i][0])

X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1, 1)

In [85]:
print(X_train.shape)
print(y_train.shape)

(261920, 24, 3)
(261920, 1)


In [99]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, 3)).reshape(-1, 24, 3)
y_train = scaler.fit_transform(y_train)

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [92]:
baseline_model = Sequential([
    # LSTM(32, return_sequences=True, input_shape=(24, 3)),
    Dense(32, input_shape=(24, 3), kernel_regularizer=l2(0.01)),
    Flatten(),
    # BatchNormalization(),
    # Dropout(0.5),
    # LSTM(32),
    Dense(32),
    # BatchNormalization(),
    # Dropout(0.5),
    Dense(1)
])

baseline_model.compile(optimizer='adam', loss='mean_squared_error')
baseline_model.summary()


/Users/darren/anaconda3/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                │ (None, 24, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 32)             │        24,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,769 (96.75 KB)

 Trainable params: 24,769 (96.75 KB)

 Non-trainable params: 0 (0.00 B)

In [93]:
baseline_model.fit(X_train, y_train, epochs=1, validation_split=0.1, verbose=1)

1859/7367 ━━━━━━━━━━━━━━━━━━━━ 5s 990us/step - loss: nan

KeyboardInterrupt: 

In [90]:
baseline_model.predict(X_train[0].reshape(1, 24, 3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


array([[nan]], dtype=float32)

In [62]:
# # Check for NaN values in the training data
# print(f"NaN values in X_train: {np.isnan(X_train).sum()}")
# print(f"NaN values in y_train: {np.isnan(y_train).sum()}")

# Fit the model
baseline_model.fit(X_train, y_train, epochs=2, validation_split=0.1, verbose=1)

# Evaluate the model
loss = baseline_model.evaluate(X_train, y_train, verbose=0)
print(f"Train Loss (MSE): {loss:.6f}")

Epoch 1/2
4930/7367 ━━━━━━━━━━━━━━━━━━━━ 1s 632us/step - loss: nan

KeyboardInterrupt: 

In [ ]:
baseline_test_data = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"] + ["bonk", "floki", "pepe", "wif"]:
    baseline_test_data.append(pd.read_csv(f"image_data/{coin}usdt/testing_output.csv"))

baseline_test_data[0]

In [ ]:
X_test, y_test = [], []
for df in baseline_test_data:
    values = df["LogReturns"].values
    for i in range(24, len(values)):
        X_test.append(values[i-24:i])
        y_test.append(values[i])

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
loss = baseline_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss (MSE): {loss:.6f}")

# Test against baseline of predicting 0
baseline_loss = np.mean(y_test ** 2)
print(f"0 Loss: {baseline_loss:.6f}")